## 기본설정 및 함수정의

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objs as go
import plotly.offline as offline
from folium.plugins import HeatMapWithTime
from plotly.subplots import make_subplots
import folium
from folium import plugins
from folium.plugins import HeatMap
from folium import FeatureGroup
import json
import math
import re
from datetime import datetime
import os
import glob
import subprocess
from bs4 import BeautifulSoup as bs
from shapely.geometry import Point, Polygon, LineString
from shapely.ops import unary_union
import geopandas as gpd
from geopandas import GeoSeries
import pyproj
from tqdm import tqdm
from keplergl import KeplerGl

# 모든 열이 생략되지 않도록 설정
pd.set_option('display.max_columns', None)

# 영등포역 위도, 경도
ydp = [37.51569886583137, 126.90768824079629]

# Point를만드는 함수
def make_point(x):
    try:
        return Point(x)
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
# Polygon을 만드는 함수
def make_pol(x):
    try:
        return Polygon(x[0])
    except:
        return Polygon(x[0][0])
    
# Linestring을 만드는 함수
def make_lin(x):
    try:
        return LineString(x)
    except:
        return LineString(x[0])

# 데이터프레임을 GeoPandas 데이터프레임으로 변환하는 함수 정의
def geo_transform(DataFrame) :
    # csv to geopandas
    # lon, lat data를 geometry로 변경
    DataFrame['lat'] = DataFrame['lat'].astype(float)
    DataFrame['lon'] = DataFrame['lon'].astype(float)
    DataFrame['geometry'] = DataFrame.apply(lambda row : Point([row['lon'], row['lat']]), axis=1) # 위도 및 경도롤 GeoPandas Point 객체로 변환
    DataFrame = gpd.GeoDataFrame(DataFrame, geometry='geometry')
    DataFrame.crs = {'init':'epsg:4326'} # geopandas 데이터프레임의 좌표계를 EPSG 4326으로 설정
    DataFrame = DataFrame.to_crs({'init':'epsg:4326'}) # 데이터프레임의 좌표계를 자체 좌표계에서 EPSG 4326으로 변환
    return DataFrame

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\geopandas\_compat.py:124: UserWarning:

The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.

C:\Users\HJK\AppData\Local\Temp\ipykernel_17108\4035485422.py:23: DeprecationWarning:

Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using

#### 영등포구 행정경계 (출처 - 통계지리정보서비스 2023년 센서스용 행정구역경계(읍면동))

In [2]:
# shp to geodataframe convert
shapefile_path = "SBJ_2406_001/_census_data_2023_bnd_dong_bnd_dong_11190_2023_2023\\bnd_dong_11190_2023_2023_2Q.shp"
ydp_gdf = gpd.read_file(shapefile_path)
ydp_gdf = ydp_gdf.to_crs(epsg=4326) #EPSG4326 형식으로 변환

#### 격자(매핑용)

In [3]:
# GeoJSON 파일 불러오기
with open('SBJ_2406_001/19.영등포_격자(매핑용).geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
grid_map_df = pd.json_normalize(geojson_data['features'])
grid_map_df['geometry'] = grid_map_df['geometry.coordinates'].apply(lambda x : make_pol(x))
grid_map_df.drop(columns="geometry.coordinates", axis=1, inplace=True)
# grid_map_df 데이터프레임을 GeoDataFrame으로 변환
grid_map_df = gpd.GeoDataFrame(grid_map_df, geometry='geometry')

#### 영등포구 상세 도로망

In [4]:
# GeoJSON 파일 불러오기
with open('SBJ_2406_001/6.영등포_상세도로망.geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
roadsystem_df = pd.json_normalize(geojson_data['features'])
roadsystem_df['geometry'] = roadsystem_df['geometry.coordinates'].apply(lambda x : make_lin(x))
roadsystem_df['properties.link_id'] = roadsystem_df['properties.link_id'].astype(str)
roadsystem_df.drop(columns="geometry.coordinates", axis=1, inplace=True)
# roadsystem_df 데이터프레임을 GeoDataFrame으로 변환
roadsystem_df = gpd.GeoDataFrame(roadsystem_df, geometry='geometry')

#### 도로명주소(도로)

In [5]:
# GeoJSON 파일 불러오기
with open('SBJ_2406_001/7.영등포_도로명주소(도로).geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
roadname_df = pd.json_normalize(geojson_data['features'])
roadname_df['geometry.coordinates'] = roadname_df['geometry.coordinates'].apply(lambda x : make_lin(x))
roadname_df.rename(columns={'geometry.coordinates' : 'geometry'}, inplace=True)
# roadname_df 데이터프레임을 GeoDataFrame으로 변환
roadname_df = gpd.GeoDataFrame(roadname_df, geometry='geometry')

#### 추정교통량 시각화

In [6]:
# 도로 id와 geometry를 매핑한 딕셔너리 생성 : 추정교통량 data와 매핑 예정
road_geo_dic = dict(zip(roadsystem_df['properties.link_id'], roadsystem_df['geometry']))

traffic_df = pd.read_csv('SBJ_2406_001/5.영등포_추정교통량.csv')
traffic_df['year'] = traffic_df['year'].astype(str)
traffic_df['link_id'] = traffic_df['link_id'].astype(str)
traffic_df['link_id'] = traffic_df['link_id'].apply(lambda x : x[:-2])
traffic_df = traffic_df[traffic_df['timeslot'] == 'all'] # 일별 누적교통량 사용
traffic_df['geometry'] = traffic_df['link_id'].map(road_geo_dic)
traffic_df = gpd.GeoDataFrame(traffic_df, geometry='geometry')
traffic_df

C:\Users\HJK\AppData\Local\Temp\ipykernel_17108\3902600021.py:4: DtypeWarning:

Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.



,year,link_id,road_rank,road_length,road_nm,sido_nm,sgg_nm,emd_nm,timeslot,ALL_AADT,PSCR_AADT,BUS_AADT,FGCR_AADT,geometry
6316,2020,478713185,102,0.155,올림픽대로,서울특별시,영등포구,당산2동,all,95932,84193,1790,9949,"LINESTRING (126.90284 37.53766, 126.90197 37.5..."
6317,2020,478713183,102,0.526,올림픽대로,서울특별시,영등포구,당산2동,all,99454,87969,1566,9918,"LINESTRING (126.90749 37.53472, 126.90724 37.5..."
6318,2020,478704859,102,0.979,올림픽대로,서울특별시,영등포구,당산2동,all,94804,82507,1487,10810,"LINESTRING (126.90155 37.53861, 126.90134 37.5..."
6319,2020,478704857,102,1.334,올림픽대로,서울특별시,영등포구,당산2동,all,93835,81792,1738,10305,"LINESTRING (126.89112 37.54443, 126.89127 37.5..."
6320,2020,478737398,102,0.007,서부간선도로,서울특별시,영등포구,문래동,all,7101,3929,129,3043,"LINESTRING (126.88194 37.51626, 126.88192 37.5..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265335,2022,478703028,104,0.096,선유동1로,서울특별시,영등포구,당산1동,all,10453,9623,83,747,"LINESTRING (126.89204 37.52474, 126.89226 37.5..."
265336,2022,478752104,104,0.023,의사당대로,서울특별시,영등포구,여의동,all,5913,5196,173,545,"LINESTRING (126.92747 37.51804, 126.92773 37.5..."
265337,2022,478742904,104,0.014,NaN,서울특별시,영등포구,문래동,all,786,455,36,295,"LINESTRING (126.89198 37.51849, 126.89183 37.5..."
265338,2022,478728772,104,0.044,양평로,서울특별시,영등포구,당산2동,all,20244,16072,2248,1923,"LINESTRING (126.90146 37.53416, 126.90104 37.5..."


In [ ]:
# 맵 객체 생성 및 데이터 로드
ydp_road_estimating_map = KeplerGl(height=1000, width=1500)
ydp_road_estimating_map.add_data(data=traffic_df, name="영등포구 추정교통량 데이터")
ydp_road_estimating_map.add_data(data=ydp_gdf, name="영등포구 행정동 데이터")
# 맵 출력 및 상세설정
ydp_road_estimating_map

In [35]:
# 맵 저장
ydp_road_estimating_map.save_to_html(file_name="visualization/영등포구 교통관련/영등포구 추정교통량 map.html")

Map saved to visualization/영등포구 교통관련/영등포구 추정교통량 map.html!


#### 교통사고이력 시각화

In [57]:
# GeoJSON 파일 불러오기
with open('SBJ_2406_001/8.영등포_교통사고이력.geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
accident_df = pd.json_normalize(geojson_data['features'])
accident_df['geometry'] = accident_df['geometry.coordinates'].apply(lambda x : make_pol(x))

# accident_df 데이터프레임을 GeoDataFrame으로 변환
accident_df = gpd.GeoDataFrame(accident_df, geometry='geometry')
accident_df.columns = [i.split("_")[-1] for i in accident_df.columns]
# 교통사고 건수 통합(3개년 통합)
accident_df['tot_acc'] = accident_df['2020'] + accident_df['2021'] + accident_df['2022']
accident_df['tot_acc'] = accident_df['tot_acc'].fillna(0)
accident_df.rename(columns={'properties.gid' : 'gid'}, inplace=True)
accident_df = accident_df[['gid', 'tot_acc', 'geometry']]
accident_df = accident_df[accident_df['tot_acc'] != 0]

In [ ]:
# 맵 객체 생성 및 데이터 로드
ydp_accident_map = KeplerGl(height=1000, width=1500)
ydp_accident_map.add_data(data=ydp_gdf, name="영등포구 행정동 데이터")
ydp_accident_map.add_data(data=roadname_df, name="영등포구 도로망 데이터")
ydp_accident_map.add_data(data=accident_df, name="영등포구 교통사고 데이터")

# 맵 출력 및 상세설정
ydp_accident_map

In [60]:
# 맵 저장
ydp_accident_map.save_to_html(file_name="visualization/영등포구 교통관련/영등포구 교통사고이력 map.html")

Map saved to visualization/영등포구 교통관련/영등포구 교통사고이력 map.html!


#### 주차민원위치정보 시각화

In [7]:
parking_petition_df = pd.read_csv('SBJ_2406_001/30.영등포_주차민원위치정보.csv')
parking_petition_df['apply_ymd'] = pd.to_datetime(parking_petition_df['apply_ymd'], format='%Y%m%d')
parking_petition_df['apply_ymd'] = parking_petition_df['apply_ymd'].astype(str)
parking_petition_df = geo_transform(parking_petition_df)
parking_petition_df

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



,apply_ymd,lon,lat,geometry
0,2021-10-31,126.926841,37.513969,POINT (126.92684 37.51397)
1,2021-11-13,126.924122,37.509871,POINT (126.92412 37.50987)
2,2021-11-13,126.924122,37.509871,POINT (126.92412 37.50987)
3,2021-11-13,126.926895,37.513969,POINT (126.92690 37.51397)
4,2021-11-13,126.902834,37.487867,POINT (126.90283 37.48787)
...,...,...,...,...
131460,2023-12-31,126.901489,37.493373,POINT (126.90149 37.49337)
131461,2023-12-31,126.909838,37.522266,POINT (126.90984 37.52227)
131462,2023-12-31,126.898187,37.496669,POINT (126.89819 37.49667)
131463,2023-12-31,126.916229,37.513346,POINT (126.91623 37.51335)


In [8]:
# 공간 조인 수행
joined = gpd.sjoin(parking_petition_df, grid_map_df, how="inner", op='within')
# 각 격자 내의 불법 주정차 점 개수 계산
grid_parking_counts = joined.groupby(joined.index_right).size()
# 결과를 grid_map_df에 추가
grid_map_df_petition = grid_map_df.copy()
grid_map_df_petition['parking_petition_count'] = grid_parking_counts
# NaN 값을 0으로 대체
grid_map_df_petition['parking_petition_count'] = grid_map_df_petition['parking_petition_count'].fillna(0)

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning:

The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.

C:\Users\HJK\AppData\Local\Temp\ipykernel_17108\3248560573.py:2: UserWarning:

CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: None




In [ ]:
# 맵 객체 생성 및 데이터 로드
ydp_parking_petition_map = KeplerGl(height=1000, width=1500)
ydp_parking_petition_map.add_data(data=ydp_gdf, name="영등포구 행정동 데이터")
ydp_parking_petition_map.add_data(data=roadname_df, name="영등포구 도로망 데이터")
ydp_parking_petition_map.add_data(data=parking_petition_df, name="영등포구 주차민원 데이터")
ydp_parking_petition_map.add_data(data=grid_map_df_petition, name="영등포구 주차민원 격자매핑 데이터")

# 맵 출력 및 상세설정
ydp_parking_petition_map

In [ ]:
# 맵 저장
ydp_parking_petition_map.save_to_html(file_name="visualization/영등포구 교통관련/영등포구 주차민원 시각화 map.html")

#### 불법주정차 단속이력 시각화

In [17]:
illegal_parking_df = pd.read_csv('SBJ_2406_001/24.영등포_불법주정차단속이력.csv')
illegal_parking_df['cd_ymd'] = pd.to_datetime(illegal_parking_df['cd_ymd'], format='%Y%m%d')
illegal_parking_df['cd_ymd'] = illegal_parking_df['cd_ymd'].astype(str)
illegal_parking_df = geo_transform(illegal_parking_df)
illegal_parking_df

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



,cd_ymd,lon,lat,geometry
0,2021-01-01,126.891659,37.532106,POINT (126.89166 37.53211)
1,2021-01-01,126.892012,37.531427,POINT (126.89201 37.53143)
2,2021-01-01,126.902804,37.500968,POINT (126.90280 37.50097)
3,2021-01-01,126.932557,37.523259,POINT (126.93256 37.52326)
4,2021-01-01,126.922281,37.505354,POINT (126.92228 37.50535)
...,...,...,...,...
549810,2023-12-31,126.926353,37.524632,POINT (126.92635 37.52463)
549811,2023-12-31,126.920194,37.530227,POINT (126.92019 37.53023)
549812,2023-12-31,126.900990,37.490421,POINT (126.90099 37.49042)
549813,2023-12-31,126.902166,37.516641,POINT (126.90217 37.51664)


In [18]:
# 공간 조인 수행
joined = gpd.sjoin(illegal_parking_df, grid_map_df, how="inner", op='within')
# 각 격자 내의 불법 주정차 점 개수 계산
grid_parking_counts = joined.groupby(joined.index_right).size()
# 결과를 grid_map_df에 추가
grid_map_df_illegal = grid_map_df.copy()
grid_map_df_illegal['illegal_parking_count'] = grid_parking_counts
# NaN 값을 0으로 대체
grid_map_df_illegal['illegal_parking_count'] = grid_map_df_illegal['illegal_parking_count'].fillna(0)

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning:

The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.

C:\Users\HJK\AppData\Local\Temp\ipykernel_42484\964203307.py:2: UserWarning:

CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: None




In [ ]:
# 맵 객체 생성 및 데이터 로드
ydp_illegal_parking_map = KeplerGl(height=1000, width=1500)
ydp_illegal_parking_map.add_data(data=ydp_gdf, name="영등포구 행정동 데이터")
ydp_illegal_parking_map.add_data(data=roadname_df, name="영등포구 도로망 데이터")
ydp_illegal_parking_map.add_data(data=illegal_parking_df, name="영등포구 불법주정차단속 데이터")
ydp_illegal_parking_map.add_data(data=grid_map_df_illegal, name="영등포구 불법주정차단속 격자매핑 데이터")

# 맵 출력 및 상세설정
ydp_illegal_parking_map

In [20]:
# 맵 저장
ydp_illegal_parking_map.save_to_html(file_name="visualization/영등포구 교통관련/영등포구 불법주정차단속이력 map.html")

Map saved to visualization/영등포구 교통관련/영등포구 불법주정차단속이력 map.html!


#### 주차장 시각화

In [38]:
# 주차장 데이터프레임을 GeoPandas 데이터프레임으로 변환
parkinglot_df = pd.read_csv('SBJ_2406_001/26.영등포_주차장현황.csv')
parkinglot_df.fillna(1, inplace=True)

In [ ]:
parkinglot_df[~parkinglot_df['slots'].isnull()]

,carpark_nm,carpark_gbn,carpark_type,slots,lon,lat,geometry
0,영등포본동제1 공영주차장,공영,노외,194.0,126.909151,37.514358,POINT (126.90915 37.51436)
1,영등포동제1 공영주차장,공영,노외,34.0,126.910054,37.520748,POINT (126.91005 37.52075)
2,영등포동제2 공영주차장,공영,노외,28.0,126.908767,37.523664,POINT (126.90877 37.52366)
3,당산1동 공영주차장,공영,노외,91.0,126.900006,37.523768,POINT (126.90001 37.52377)
4,당산2동 공영주차장,공영,노외,31.0,126.907755,37.528254,POINT (126.90775 37.52825)
...,...,...,...,...,...,...,...
4415,영등포동 10구역,거주자우선,구간,9.0,126.914659,37.517869,POINT (126.91466 37.51787)
4416,영등포동 10구역,거주자우선,구간,9.0,126.914574,37.517889,POINT (126.91457 37.51789)
4417,영등포동 10구역,거주자우선,구간,9.0,126.914399,37.517929,POINT (126.91440 37.51793)
4418,영등포동 10구역,거주자우선,구간,9.0,126.914348,37.517936,POINT (126.91435 37.51794)


In [ ]:
# 맵 객체 생성 및 데이터 로드
ydp_parkinglot_map = KeplerGl(height=1000, width=1500)
ydp_parkinglot_map.add_data(data=parkinglot_df, name="영등포구 주차장 데이터")
ydp_parkinglot_map.add_data(data=ydp_gdf, name="영등포구 행정동 데이터")
ydp_parkinglot_map.add_data(data=roadname_df, name="영등포구 도로망 데이터")

# 맵 출력 및 상세설정
ydp_parkinglot_map

In [ ]:
ydp_parkinglot_map

In [42]:
# 맵 저장
ydp_parkinglot_map.save_to_html(file_name="visualization/영등포구 교통관련/영등포구 주차장 map.html")

Map saved to visualization/영등포구 교통관련/영등포구 주차장 map.html!
